<style>
th {background-color:#55FF33;}
td {background-color:#00FFFF;}
</style>

<img align="right" style="max-width: 200px; height: auto" src="./assets/logo.png">

## Lab 04 - Statistische Analyseverfahren

Lehrgang Internal Auditing, Universität St.Gallen (HSG), 2022

Die Analysen des Seminars **Audit Data Analytics** basieren auf Jupyter Notebook (https://jupyter.org). Anhand solcher Notebooks ist es möglich eine Vielzahl von Datenanalysen und statistischen Validierungen durchzuführen.

<img align="center" style="max-width: 900px; height: auto" src="./assets/banner.png">

Im Rahmen des vorhergehenden Labs haben wir einige der grundlegenden Vorgehensweisen der **Regelbasierter Analyseverfahren** kennengelernt. In diesem Lab werden wir Jupyter Notebook verwenden, um die im Seminar vorgestellten **Statistischen Analyseverfahren** praktisch zu vertiefen. Das Hauptziel dieses Notebook ist es, die einzelnen Schritte solcher Analyseverfahren anhand des Beispiels der **Newcomb-Benford Analyse** erfasster Zahlungen durchzuführen.

<img align="center" style="max-width: 800px; height: auto" src="./assets/analytics_process.png">

Im Zweifelsfall oder bei Fragen wenden Sie sich, wie immer gerne an uns via **marco (dot) schreyer (at) unisg (dot) ch**. Wir wünschen Ihnen Viel Freude mit unseren Notebooks und Ihren forensischen Analysen!

## Lernziele des Labs:

Nach der heutigen Lab sollten Sie in der Lage sein:

> 1. Erste eigene Newcomb-Benford Datenanlysen mit **Jupyter** und **Python** durchzuführen.
> 2. Die Bibliotheken **Pandas** und **Matplotlib** im Kontext Statistischer Analyseverfahren anzuwenden.
> 3. Buchungen im Hinblick auf **ungewöhnliche Betragsverteilungen** statistisch zu analysieren.
> 4. Erste **konkrete Ideen** für mögliche Statistische Datenanalysen in Ihrem Unternehmen zu entwickeln.

## 1. Einrichtung der Jupyter Notebook-Umgebung

In Analogie zu unserem einführenden Notebook ist es zunächst wieder notwendig, einige Python-Bibliotheken zu importieren, die es uns ermöglichen, Daten zu importieren, zu analysieren und zu visualisieren. In diesem Notebook werden wir hierzu wieder die in **Lab 01** vorgestellten Bibliotheken **(1) Pandas** (https://pandas.pydata.org), **(2) NumPy** (https://numpy.org) und **(3) Matplotlib** (https://matplotlib.org) verwenden.

Lassen Sie uns nun die beiden wichtigsten Datenanalyse-Bibliotheken `Pandas` und `NumPy` entsprechend importieren, indem wir die nachfolgenden beiden `import` Anweisungen ausführen:

In [ ]:
import pandas as pd
import numpy as np

# setzen globaler Pandas Parameter
pd.options.display.max_rows = 500 # allgemeine Darstellung Anzahl Zeilen
pd.options.display.float_format = '{:.2f}'.format # numerische Darstellung von Gleitkommazahlen

Ausserdem importieren wir einige **Utility Bibliotheken**, d.h. Bibliotheken die wichtige zusätzliche Funktionalität zur Verfügung stellen:

In [ ]:
import os # ermöglicht die Erstellung, den Zugriff und die Manipulation von Datenverzeichnissen
import datetime as dt # ermöglicht die Erstellung von Zeitstempeln für Daten

Auch importieren wir eine Reihe von **zusätzlichen Bibliotheken** für den Datenzugriff und den Datenimport in Python:

In [ ]:
import io # ermöglicht das Öffnen und den Zugriff auf Datenströme
import pathlib # ermöglicht das Schreiben von lokaken Dateien
import zipfile # ermöglicht das packen und ent-packen von Zip Archiven
import urllib.request # ermöglicht die Erstellung von Webseiten Anfragen

Schliesslich importieren wir wieder die Bibliothek `Matplotlib` und setzen einige der allgemeinen Parameter für die Datenvisualisierung:

In [ ]:
import matplotlib.pyplot as plt

# setzen globaler Matplotlib Paramater der Datenvisualisierung
plt.style.use('seaborn') # den Visualisierungsstil festlegen
plt.rcParams['figure.figsize'] = [5, 5] # die Visualisierungsgrösse festlegen
plt.rcParams['figure.dpi']= 150 # die Visualisierungsauflösung festlegen

Die nachfolgende Anweisung aktiviert das sog. **Inline-Plotten** von Schaubildern innerhalb des aktuellen Notebooks:

In [ ]:
%matplotlib inline

Darüber hinaus ist es gute Praxis, für jedes Notebook eine entsprechende Ordnerstruktur anzulegen. Diese Ordnerstruktur dient im weiteren Analyseverlauf dazu, sowohl die **Originaldaten** als auch **Analyseergebnisse** und **Schaubilder** zu speichern:

In [ ]:
# erstellen des Verzeichnis der Orignaldaten
original_data_dir = './01_original_data'
if not os.path.exists(original_data_dir): os.makedirs(original_data_dir) 

# erstellen des Verzeichnis der Analysedaten
analysis_data_dir = './02_analysis_data'
if not os.path.exists(analysis_data_dir): os.makedirs(analysis_data_dir)

Abschliessend möchten wir wieder mögliche **Warnungen** einzelner Bibliotheken ignorieren:

In [ ]:
import warnings # ermöglicht die Handhabung von Warnmeldungen

# setzen des Warnfilter-Flags, um Warnungen zu ignorieren
warnings.filterwarnings('ignore')

Solche Warnungen können oftmals aufgrund von aktuellen Bibliothekserweiterungen bzw. -weiterentwicklungen erscheinen. Diese sollen uns jedoch im weiteren Analyseverlauf nicht stören.

## 2. Importieren, Validieren und Aufbereiten des PaySim Datensatzes

Der synthetische **PaySim**-Datensatz simuliert mobile Finanztransaktionen eines auf dem afrikanischen Kontinent tätigen Anbieters von mobilen Finanzdienstleistungen. Die Transaktionen entsprechen den realen Transaktionen eines Monats und wurden durch den Dienstleister im Rahmen einer **Kaggle Competition** zur Verfügung gestellt: https://www.kaggle.com/ntnu-testimon/paysim1.

Zum Zeitpunkt der Veröffentlichung der Daten war der Dienstleister in 14 Ländern weltweit tätig. Die aktuelle Version des Datensatzes wurde am 3. April 2017 durch die **Norwegischen Universität für Wissenschaft und Technologie (NTNU)** veröffentlicht.

Insgesamt umfasst der **PaySim**-Datensatz eine Population von **6,3 Millionen protokollierten Transaktionen**. Jede Transaktion enthält **neun verschiedene Attribute (Merkmale)**. Die Attributnamen und ihre jeweilige semantische Bedeutung sind im Folgenden aufgeführt:

>- `Step:` Bezeichnet die aktuelle Stunde der Zeit. Insgesamt 744 Stunden (30 Simulationstage).
>- `Typ:` Bezeichnet die Art der Transaktion. Insgesamt 5 verschiedene Transaktionsarten.
>- `Betrag:` Bezeichnet den überwiesenen Geldbetrag der Transaktion in Landeswährung.

>- `NameOrig:` Bezeichnet die (anonymisierte) ID des Absenders, der die Transaktion in Auftrag gegeben hat.
>- `OldBalanceOrg:` Bezeichnet den Anfangssaldo des Kontos des Absenders vor der Transaktion.
>- `NewBalanceOrg:` Bezeichnet den neuen Saldo des Kontos des Absenders nach der Transaktion.

>- `NameDest:` Bezeichnet die (anonymisierte) ID des Empfängers der Transaktion.
>- `OldBalanceOrg:` Bezeichnet den ursprünglichen Kontostand des Empfängers vor der Transaktion.
>- `NewBalanceOrg:` Bezeichnet den neuen Kontostand des Empfängers nach Durchführung der Transaktion.

Darüber hinaus ist jede Transaktion durch die folgenden **zwei zusätzlichen Merkmale ("Flags")** gekennzeichnet:

>- `isFraud:` Kennzeichnet eine tatsächlich "betrügerische" Transaktionen.
>- `isFlaggedFraud:` Kennzeichnet ein durch das System erkannte "betrügerische" Transaktionen.

### 2.1. Download the PaySim Dataset of Financial Transactions

In einem ersten Schritt importieren wir nun einen Teilauszug des zuvor beschriebenen Datensatzes, bestehend aus **2.770.409 Zahlungstransaktionen**, in das Notebook. Hierzu ist es zunächst wieder notwendig, den Pfad bzw. die Internetadressen der zu importierenden Tabellen zu definieren:

In [ ]:
url = 'https://raw.githubusercontent.com/GitiHubi/courseACA/master/lab04/data/transactions.zip'

Weitere Einzelheiten über diesen Datensatz sind, in der nachfolgenden Veröffentlichung zu finden: *E. A. Lopez-Rojas , A. Elmir, und S. Axelsson. "PaySim: A financial mobile money simulator for fraud detection". In: The 28th European Modeling and Simulation Symposium-EMSS, Larnaca, Cyprus. 2016* 

Im Sinne guter forensischer Praxis und den damit verbundenen Dokumentationszwecken erzeugen wir auch einen **Zeitstempel des Datendownloads**. Die Erstellung des Zeitstempels erfolgt über die `utcnow` Anweisung ([Dokumentation](https://docs.python.org/3/library/datetime.html)) der `datetime` Bibliothek und Formatangabe des Zeitstempels:

In [ ]:
# definition des Zeitstempels des Datendownloads
timestamp = dt.datetime.utcnow().strftime('%Y-%m-%d_%H-%M-%S')

In einem nächsten Schritt öffnen wir eine Verbindung zu der zuvor definierten Internetadresse (URL) des PaySim Datensatzes. Hierzu verwenden wir die Anweisung `request.urlopen` der bereits importierten `URLLib` Bibliothek:

In [ ]:
paysim_request = urllib.request.urlopen(url) # erstellen einer Verbindung zur PaySim URL

Im Anschluss werden die Transaktionsdaten **unverändert** heruntergeladen. D.h. in einem ersten Schritt werden die Daten im sogenannten **Byte-Format** heruntergeladen. In einem zweiten Schritt dann anschliessend als **Zip-Archiv** interpretiert:

In [ ]:
# download der PaySim Transaktionsdaten im Byte-Format
data_raw = io.BytesIO(paysim_request.read()) # einlesen der byte Daten

# interpretation der PaySim Transaktionsdaten als Zip-Archiv
data_zip = zipfile.ZipFile(data_raw) # einlesen der zip Daten

Um die Daten im CSV-Format speichern bzw. bearbeiten zu könnnen ist es zunächst notwendig die Daten aus dem zuvor erhaltenen **Zip-Archiv zu extrahieren**. Die Extraktion erfolgt anhand der `open` Anweisung ([Dokumentation](https://docs.python.org/3/library/zipfile.html)) der `zipfile` Bibliothek:

In [ ]:
# öffnen und extraktion der innerhalb des Zip-Archivs enthaltenen Daten
data_csv = data_zip.open('transactions.csv')

In einem nächsten Schritt möchten wir die PaySim Transaktionsdaten als `Pandas` DataFrame in das Notebook importieren. Hierzu ist es notwendig, die beiden im CSV-Format vorliegenden Dateien über die `read_csv` Anweisung ([Dokumentation](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html)) der `Pandas` Bibliothek einzulesen:

In [ ]:
transactions = pd.read_csv(data_csv, sep=',', index_col=0, thousands=',') # einlesen der PaySim Daten 

Lassen Sie uns nun auch die **5 ersten Zeilen** der **PaySim Transaktionen** anschauen um zu überprüfen, dass die Daten grundsätzlich im richtigen Format d.h. als `Pandas` DataFrame importiert wurden. Hierzu verwenden wir wieder die `head` Anweisung ([Dokumentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html)) der `Pandas` Bibliothek: 

In [ ]:
transactions.head(5)

Nachfolgend lassen Sie uns nun auch die **5 letzten Zeilen** der **PaySim Transaktionen** anschauen um zu überprüfen, dass die Daten grundsätzlich im richtigen Format d.h. als `Pandas` DataFrame importiert wurden. Hierzu verwenden wir wieder die `tail` Anweisung ([Dokumentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.tail.html)) der `Pandas` Bibliothek: 

In [ ]:
transactions.tail(5)

Abschliessend speichern wir die Daten erneut auf dem lokalen Dateisystem im CSV-Format. Der Speichervorgang erfolgt innerhalb des bereits zuvor erstellten Ordners für zu **analysierende Originaldaten** mit dem Namen ***02_analysis_data***. Für den tatsächlichen Export der Transaktionen wieder kann auf die `to_csv` Anweisung ([Dokumentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html)) der `Pandas` Bibliothek zurückgegriffen werden:

In [ ]:
# definition des Dateinamens der PaySim Analysedaten
paysim_analysis_file_name = timestamp + '_paysim_file_analysis.csv'

# definition Dateipfades der PaySim Originaldaten
paysim_path_analysis = os.path.join(analysis_data_dir, paysim_analysis_file_name)

# speichern der PaySim Originaldaten
transactions.to_csv(paysim_path_analysis, header=True, index=False, encoding='utf-8')

### 2.2. Validieren der Importierten Daten

In einem nächsten Schritt validieren wir, wie zuvor in **Lab 02**, die Vollständigkeit der importierten Daten. Hierzu ermitteln wir die Anzahl der Datenzeilen und Datenspalten und gleichen diese Information mit der erwarteten Zeilen- und Spaltenanzahl ab. Der Abgleich erfolgt unter Verwendung der `shape` Anweisung ([Dokumentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.shape.html)) der `Pandas`Bibliothek.  

Lassen Sie uns hierzu die Dimensionalität der Tabelle von **PaySim Transaktionen** auswerten:

In [ ]:
transactions.shape

Die eingelesene Tabelle der PaySim Transaktionen umfasst insgesamt **2.770.409 Zeilen** und **10 Spalten**. Um die Vollständigkeit der erhaltenen Daten zu gewährleisten gilt es wieder, diese Informationen mit der erhaltenen Datendokumentation abzugleichen.

## 3. Aufbereitung der Importierten Daten

**Data preparation** defines the cleaning and transformation of raw data prior to the actual processing and analysis. Data preparation is an important step before the actual data analysis to be performed and often involves reformatting data, correcting information, and combining data sets to enrich that data.

### 3.1. Erstellung eines eindeutigen Identifikationsmerkmals

Lassen Sie uns nun zunächst, **wie zuvor in Lab 02**, einen eindeutigen Schlüssel (Identifkationsmerkmal) für die einzelnen Transaktionen der PaySim Tabelle erstellen. Das **Identifikationsmerkmal** wird innerhalb des Datensatzes zur eindeutigen Kennzeichnung einzelner Transaktionen verwendet. Das Erstellen einer solchen Merkmals kann bspw. durch eine Folge von Werten, z.B. einer fortlaufenden ein-eindeutige Nummernfolge, erfolgen. 

Im Folgenden erzeugen wir eine solche **eindeutige Folge von Identifikationsmerkmalen**, nachfolgend "KEY's" (Schlüssel) genannt, unter Verwendung der  Namenskonvention `ACA_ID_0000001`, `ACA_ID_0000002`,..., `ACA_ID_2770408` erstellen. Das Erstellen einer **solchen Folge von KEY's** erfolgt unter Verwendung der `zfill` Anweisung ([Dokumentation](https://docs.python.org/3/library/stdtypes.html)) des `String` Datentyps in `Python`.

In [ ]:
# erstellen einer liste fortlaufender numerischer werte 0, 1, 2, ..., N
ids = list(range(0, data.shape[0]))

# erstellen einer liste ein-eindeutiger identifikationsmerkmale
keys = ['ACA_ID_' + str(e).zfill(7) for e in ids]

Im Anschluss überprüfen wir die **ersten acht** erstellten ein-eindeutigen KEY's:

In [ ]:
keys[0:8]

In einem nächsten Schritt fügen wir die erstellten ein-eindeutigen Identifikationsmerkmake **KEYS** den einzelnen Transaktionen des PaySim Datensatzes jeweils hinzu. Dies erfolgt durch das Einfügen einer zusätzlichen Spalte in den PaySim Datensatz von Transaktionen. Die Spalte soll die Spaltenbezeichnung **KEY** aufweisen und die führende Spalte des `Pandas` Dataframes darstellen. Um dies zu gewährleisten, erfolgt das Einfügen unter Verwendung der `insert` Anweisung ([Dokumentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.insert.html)) der `Pandas` Bibliothek:

In [ ]:
transactions.insert(0, 'KEY', keys)

Lassen Sie uns nun überprüfen ob die Spalte **KEY** erfolgreich erstellt wurde, indem wir die **ersten 10 Zeilen** des PaySim Datensatzes anschauen:

In [ ]:
transactions.head(10)

Lassen Sie uns die erstellte Spalte nun auch anhand der **10 letzten Zeilen** der Tabelle des PaySim Datensatzes überprüfen:

In [ ]:
transactions.tail(10)

### 3.2. Extraktion der Zahlungs-Transaktionen 'CASH_OUT'

Nachfolgend extrahieren wir alle innerhalb des PaySim Datensatzes enthaltenen Zahlungen. Innerhalb des Datensatzes sind Zahlungen durch den Transaktionstyp **CASH_OUT** des Attributes `type` gekennzeichnet. Um die entsprechenden Zahlungen zu extrahieren nutzen wir die durch die `Pandas`Bibliothek zur Verfügung gestellten Möglichkeiten des Filterns von Daten:

In [ ]:
# filtern des datensatzes nach CASH_OUT transaktionen
transactions_cash_out = transactions[transactions["type"] == "CASH_OUT"]

Lassen Sie uns nun auch die **Dimensionalität der extrahierten Zahlungstransaktionen** auswerten. Der Auswertung erfolgt unter Verwendung der `shape` Anweisung ([Dokumentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.shape.html)) der `Pandas`Bibliothek. 

In [ ]:
transactions_cash_out.shape

Die extrahierten Zahlungstransaktionen umfassen **2.237.500 Zeilen** und **11 Spalten** (d.h. inkl. der zuvor hinzugefügten KEY Spalte). Anschliessend prüfen wir wieder beispielhaft die **10 ersten Zeilen** der extrahierten Zahlungen entsprechend und prüfen ob das Attribute `type` lediglich den Wert **CASH_OUT** aufweist:

In [ ]:
transactions_cash_out.head(10)

### 3.3. Formatting der beiden Fraud-Flags 'isFraud' und 'isFlaggedFraud'

Lassen Sie uns eine einfache **semantische Formatierung der Attribute** `isFraud` und `isFlaggedFraud` vornehmen, um die Interpretierbarkeit der Attribute für einen menschlichen Prüfer zu vereinfachen. Hierzu überprüfen wir zunächst die aktuelle Formatierung beider Attribute, indem wir die ersten fünf Zeilen der Zahlungen des PaySim Datensatzes anschauen: 

In [ ]:
transactions_cash_out.head(5)

Es ist zu beobachten, dass das Attribut "isFraud" zwei binäre Werte umfasst. Die Werte entsprechen entweder dem Wert `1`, der eine betrügerische Transaktion kennzeichnet, oder dem Wert `0`, der eine nicht betrügerische Transaktion kennzeichnet. In einem nächsten Schritt werden wir diese Werte im Datensatz neu formatieren. Die Formatierung erfolgt unter Verwendung der `iloc` Anweisung ([Dokumentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iloc.html)) der `Pandas` Bibliothek:

In [ ]:
# filtern fraudbehafteter transaktionen und ersetzen des "isFraud" flags
transactions_cash_out.loc[transactions_cash_out['isFraud'] == 1, 'isFraud'] = 'yes' # ersetzen des Wertes "1" durch "yes"

# filtern nicht-fraudbehafteter transaktionen und ersetzen des "isFraud" flags
transactions_cash_out.loc[transactions_cash_out['isFraud'] == 0, 'isFraud'] = 'no' # ersetzen des Wertes "0" durch "no"

Nun überprüfen wir stichprobenartig die vorgenommene Ersetzung, indem wir erneut die ersten **fünf Zeilen**, d.h. im Besonderen das `isFraud` Flag, anschauen:

In [ ]:
transactions_cash_out.head(5)

Lassen Sie uns nun die gleiche Umformatierung auf das Attribut `isFlaggedFraud` im Datensatz anwenden. Zur Erinnerung: Die Werte entsprechen entweder dem Wert `1` und kennzeichnen eine Transaktion die als betrügerisch erkannt wurde. Alternativ entsprechen die Werte dem Wert `0` und kennzeichnen eine Transaktion die nicht als betrügerisch erkannt wurde. 

In [ ]:
# filtern der als fraudbehaftet markierten transaktionen und ersetzen des "isFlaggedFraud" flags
transactions_cash_out.loc[transactions_cash_out['isFlaggedFraud'] == 1, 'isFlaggedFraud'] = 'yes' # ersetzen des Wertes "1" durch "yes"

# filtern der als nicht fraudbehaftet markierten transaktionen und ersetzen des "isFlaggedFraud" flags
transactions_cash_out.loc[transactions_cash_out['isFlaggedFraud'] == 0, 'isFlaggedFraud'] = 'no' # ersetzen des Wertes "0" durch "no"

Nun überprüfen wir stichprobenartig die vorgenommene Ersetzung, indem wir erneut die ersten **fünf Zeilen**, d.h. im Besonderen das `isFlaggedFraud` Flag, anschauen:

In [ ]:
transactions_cash_out.head(5)

## 4. Mathematisch-Statistische Analyseverfahren

Im Rahmen der revisorischen Analyse strukturierter betriebswirtschaftlicher Daten lassen sich grundlegend die nachfolgenden drei Arten von Analyseverfahren unterschieden: **(1) regelbasierte bzw. 'Red-Flag' Analysen**, **(2) Mathematisch-Statistische Analysen** und **(3) Künstliche Intelligenz bzw. Machine Learning Analysen**. Eine schematische Unterscheidung der verschiedenen Analysearten ist innerhalb des nachfolgenden Schaubilds dargestellt:

<img align="center" style="max-width: 800px; height: auto" src="./assets/analytics_overview.png">

Die verschiedenen Analysearten unterscheiden sich im Wesentlichen anhand ihrer jeweiligen Erwartungshaltung an das Analyseergebnis und das jeweils notwendige Domain oder Analyse Know-how. 

Im Kontext der in diesem Lab betrachteten **Mathematisch-Statistischen Analyseverfahren** verfügen Analyst:innen über eine im Vergleich zu **Regelbasierten Analyseverfahren** eine geringere Erwartungshaltung bzw. Hypothese an das erwartete Analyseergebnis. Zum Beispiel, das sich die kreditorischen Rechnungsbuchungen des Einkaufs üblicherweise im Intervall zwischen CHF 1.000 und CHF 40.000 bewegen. 

Darüber hinaus erfordern Mathematisch-Statistische Analyseverfahren ein **geringeres Domain Know-how** für die Umsetzung der Analyse. Jedoch erfordert die Durchführung solcher Analysen ein **höheres Datenanalyse Know-How**, beispielsweise über geeignete mathematische Verteilungen und die Berechnung statistischer Kennzahlen. 

Nachfolgend möchten wir nun beispielhaft eine mathematisch-statistische Analyse implementieren. Hierbei handelt es sich um die sog. Newcomb-Benford Analyse auf Grundlage der Zahlungsbeträge. Um eine solche Analyse zu durchzuführen ist es zunächst notwendig das durch die Analyse **(1) adressierte Risko**, **(2) die analysierte Hypothese**, **(3) die notwendigen Daten** und **(4) das Analysevorgehen** zu definieren. Die Definition der verschiedenen Analyseaspekte erfolgte innerhalb der nachfolgenden Tabelle:

<p>
    <table class="table table-bordered table-striped table-hover">
    <col width="150">
        <tr>
            <td><b align="left" style="font-size:16px">ID: </b></td>
            <td><p align="left" style="font-size:16px">Analyse 001</p></td>
        <tr>
            <td><b align="left" style="font-size:16px">Name: </b></td>
            <td><p align="left" style="font-size:16px">Analyse ungewöhnlicher Zahlungsbeträge nach Newcomb-Benford.</p></td>
        <tr>
            <td><b align="left" style="font-size:16px">Risiko: </b></td>
            <td><p align="left" style="font-size:16px">Die ungewöhnliche Häufigkeit führender Zahlungsbetragsziffern können Hinweise auf abweichende Geschäftsvorfälle, Einmalsachverhalte und dolose Handlungen darstellen. Täter*innen möchten im Rahmen der Ausübung doloser Handlungen unerkannt bleiben bzw. diese innerhalb gewöhnlicher Zahlungen verschleiern.</p></td>
        </tr>
        <tr>
            <td><b align="left" style="font-size:16px">Hypothese: </b></td>
            <td><p align="left" style="font-size:16px">Führende Ziffern der Beträge regulärer Zahlungen sind Newcomb-Benford verteilt.</p></td>
        </tr>
        <tr>
            <td><b align="left" style="font-size:16px">Data: </b></td>
            <td><p align="left" style="font-size:16px">Cash-Out Transaktionen des PaySim Datensatzes.</p></td>
        </tr>
        <tr>
            <td><b align="left" style="font-size:16px">Vorgehen: </b></td>
            <td><p align="left" style="font-size:16px">Ermittlung der Häufigkeitsverteilungen der jeweils führenden Ziffern in den Zahlungen. Abgleich der Verteilungsanalyse mit der erwarteten Newcomb-Benford Verteilung. Durchführen einer Detailprüfung möglicher Abweichungen.</p></td>
        </tr>
    </table>
</p>

### 4.1. Newcomb-Benford Analyse der führenden Ziffer des Zahlungsbetrags

In einem ersten Schritt erstellen wir eine Referenztabelle der Benford-Verteilung für jeden möglichen Wert einer einzelnen führenden Ziffer. Dazu leiten wir die Wahrscheinlichkeiten $p(d)$ nach Benford für die einzelnen führenden Ziffern ab, die durch definiert sind: 

$$ p(d) = \log_{10}(d+1) - \log_{10}(d);$$

wobei $d \in [0, 1, ...,9]$ einen tatsächlichen Wert einer führenden Ziffer bezeichnet.

Quelle: „The Law of Anomalous Numbers“, Benford F., Proceedings of the American Philosophical Society, Vol. 78, 1938, USA

#### 4.1.1 Erstellen der Newcomb-Benford Referenztabelle der führenden Ziffer

Beginnen wir mit der Erstellung eines `Pandas` Dataframes, das alle möglichen führenden Ziffern enthält: 

In [ ]:
benford_table = pd.DataFrame({"digit_1": range(1, 10)})

In einem nächsten Schritt werden wir die Wahrscheinlichkeit der Beobachtung einer bestimmten führenden Ziffer nach Benford ableiten und die Wahrscheinlichkeit entsprechend dem Dataframe hinzufügen: 

In [ ]:
benford_table["benford"] = (np.log10(benford_table["digit_1"] + 1)) - np.log10(benford_table["digit_1"])

Sehen wir uns nun die von uns erstellte Benford-Wahrscheinlichkeitstabelle der führenden Ziffern des Transaktionsbetrags an:

In [ ]:
benford_table

Außerdem berechnen wir Konfidenzintervalle von $\sigma=3$ Standardabweichungen und fügen sie der erstellten Benford-Wahrscheinlichkeitstabelle hinzu:

In [ ]:
# ermittle die absolute anzahl von cash-out transaktionen
n = transactions_cash_out.shape[0]

# ermittle die upper bound des drei
benford_table["benford_upp"] = benford_table["benford"] + 1.96 * np.sqrt((benford_table["benford"] * (1 - benford_table["benford"]))/n) 

# determine the lower bound of the three sigma confidence interval
benford_table["benford_low"] = benford_table["benford"] - 1.96 * np.sqrt((benford_table["benford"] * (1 - benford_table["benford"]))/n) 

Im Folgenden überprüfen wir die hinzugefügte untere und obere Grenze der Konfidenzintervalle:

In [ ]:
benford_table

Schliesslich wollen wir auch die erwartete Wahrscheinlichkeit der ersten führenden Ziffer nach Benford darstellen:

In [ ]:
# initialise the plot 
fig, ax = plt.subplots(figsize=(15, 5))

# plot the benford probabilities 
plt.plot(benford_table["digit_1"], benford_table["benford"], color="red")

# plot the benford probability density
plt.fill_between(benford_table["digit_1"], benford_table["benford"], color="red", alpha=0.1)

# add the axis labels
plt.ylabel("[Probability]", fontsize=12)
plt.xlabel("[Leading Digit]", fontsize=12)

# rotate x-axis tick labels
plt.xticks(rotation=0)

# add the plot title
plt.title("Benford-Newcomb Distribution - First Leading Digit", fontsize=12);

#### 4.1.2 Ermittlung tatsächlicher Auftrittswahrscheinlichkeiten der führenden Ziffer

Nachdem wir nun unsere Referenztabelle mit den Konfidenzintervallen vorbereitet haben, wollen wir uns auf die führenden Ziffern der Zahlungen konzentrieren. Dazu extrahieren wir die führenden Ziffern der einzelnen Zahlungen und fügen sie als separate Spalte dem Dataframe aller Zahlungen hinzu:

In [ ]:
transactions_cash_out["digit_1"] = transactions_cash_out["amount"].astype(str).str[0]

Anschliessend überprüfen wir die extrahierten führenden Ziffern der Zahlungen anhand der **ersten 10 Zeilen** des Zahlungen Dataframes: 

In [ ]:
transactions_cash_out[["amount", "digit_1"]].head(10)

In einem nächsten Schritt wollen wir die tatsächliche Wahrscheinlichkeit bestimmen, eine bestimmte führende Ziffer im Datensatz der Zahlungen zu beobachten. Dazu werden wir eine Liste aller beobachtbaren führenden Ziffern im Datensatz erstellen:  

In [ ]:
benford_analysis = pd.DataFrame({"digit_1": transactions_cash_out["digit_1"].value_counts().index.astype(np.int64).tolist()})

Als nächstes wird gezählt, wie oft eine bestimmte führende Ziffer in den Zahlungen vorkommt:

In [ ]:
benford_analysis["count"] = transactions_cash_out["digit_1"].value_counts().tolist()

Schließlich berechnen wir die Wahrscheinlichkeit, eine bestimmte führende Ziffer in den Zahlungen zu beobachten:

In [ ]:
benford_analysis["probability"] = benford_analysis["count"] / transactions_cash_out.shape[0]

Lassen Sie uns nun die abgeleiteten Wahrscheinlichkeiten anschauen und prüfen:

In [ ]:
benford_analysis

#### 4.1.3 Benford-Newcomb Soll vs. Ist Abweichungsanalyse der führenden Ziffer

Um die Benford-Analyse abzuschließen, wollen wir die ursprünglich erstellte Referenztabelle der Benford-Wahrscheinlichkeiten mit der tatsächlich beobachteten Wahrscheinlichkeit einer bestimmten führenden Ziffer zusammenführen. Dazu verwenden wir die Funktion `merge` aus der `Pandas` Bibliothek: 

In [ ]:
analysis_result_single_leding_digit = benford_table.merge(benford_analysis, on="digit_1")

Nun sind wir endlich in der Lage, beide Wahrscheinlichkeiten (die erwartete Wahrscheinlichkeit nach Benford-Newcomb und die beobachtete Wahrscheinlichkeit im Datensatz) zu vergleichen und mögliche Abweichungen zu erkennen: 

In [ ]:
analysis_result_single_leding_digit 

Außerdem können wir die von Benford-Newcomb erwartete Wahrscheinlichkeitsverteilung und die beobachteten Wahrscheinlichkeiten, die im Datensatz der Zahlungen verfügbar sind, visuell überprüfen:

In [ ]:
# initialise the plot 
fig, ax = plt.subplots(figsize=(15, 5))

# plot the benford probabilities 
plt.plot(analysis_result_single_leding_digit["digit_1"], analysis_result_single_leding_digit["benford"], color="red")

# plot the actual distribution of the first digit
plt.bar(analysis_result_single_leding_digit["digit_1"], analysis_result_single_leding_digit["probability"], color="green")

# plot the benford probability density
plt.fill_between(np.arange(1.0, 10.0, 1.0), analysis_result_single_leding_digit["benford"], color="red", alpha=0.1)

# add the axis labels
plt.ylabel("[Probability]", fontsize=12)
plt.xlabel("[Leading Digit]", fontsize=12)

# format the x-tick labels
plt.xticks(range(1,10), range(1,10))

# add the plot title
plt.title("Benford-Newcomb Analysis - First Leading Digit", fontsize=12);

### 4.2. Newcomb-Benford Analyse der führenden beiden Ziffern des Zahlungsbetrags

#### 4.2.1 Erstellen der Newcomb-Benford Referenztabelle der führenden beiden Ziffern

Beginnen wir wieder mit der Erstellung eines `Pandas` Dataframes, der alle möglichen Kombinationen der ersten und zweiten führenden Ziffern des Transaktionsbetrags enthält:

In [ ]:
benford_table = pd.DataFrame({"digit_2": range(1, 100)})

In ähnlicher Weise wie zuvor leiten wir die Wahrscheinlichkeit der Beobachtung einer bestimmten Kombination führender Ziffern nach Benford her. Anschliessend fügen wir die erhaltenen Wahrscheinlichkeiten dem Datenrahmen hinzu:

In [ ]:
benford_table["benford"] = (np.log10(benford_table["digit_2"] + 1)) - np.log10(benford_table["digit_2"])

Sehen wir uns nun die einzelnen Zeilen der erstellten Benford-Wahrscheinlichkeitstabelle an. Die Tabelle enthält alle möglichen Kombinationen von zwei führenden Ziffern sowie die entsprechenden Eintrittswahrscheinlichkeiten nach Benford:

In [ ]:
benford_table

Ausserdem berechnen wir Konfidenzintervalle von σ = 3 Standardabweichungen und fügen sie hinzu. Wir fügen die obere und untere Grenze der ermittelten Konfidenzintervalle in die erstellte Referenztabelle der Benford-Wahrscheinlichkeiten ein:

In [ ]:
# determine the total number of cash out transactions
n = transactions_cash_out.shape[0]

# determine the upper bound of the three sigma confidence interval
benford_table["benford_upp"] = benford_table["benford"] + 1.96 * np.sqrt((benford_table["benford"] * (1 - benford_table["benford"]))/n) 

# determine the lower bound of the three sigma confidence interval
benford_table["benford_low"] = benford_table["benford"] - 1.96 * np.sqrt((benford_table["benford"] * (1 - benford_table["benford"]))/n) 

Im Folgenden überprüfen wir die hinzugefügte untere und obere Grenze der Konfidenzintervalle:

In [ ]:
benford_table

Schließlich wollen wir auch die erwartete Wahrscheinlichkeit der ersten führenden Ziffer nach Benford darstellen:

In [ ]:
# initialise the plot 
fig, ax = plt.subplots(figsize=(15, 5))

# plot the benford probabilities 
plt.plot(benford_table["digit_2"], benford_table["benford"], color="red")

# plot the benford probability density
plt.fill_between(benford_table["digit_2"], benford_table["benford"], color="red", alpha=0.1)

# add the axis labels
plt.ylabel("[Probability]", fontsize=12)
plt.xlabel("[Leading Digits]", fontsize=12)

# format the x-tick labels
plt.xticks(range(10, 100), range(10, 100))

# rotate x-axis tick labels
plt.xticks(rotation=90)

# format the x-axis limits
plt.xlim(10, 99)

# format the y-axis limits
plt.ylim(0.0, 0.05)

# add the plot title
plt.title("Benford-Newcomb Distribution - First and Second Leading Digit", fontsize=12);

#### 4.2.2 Ermittlung tatsächlicher Auftrittswahrscheinlichkeiten der führenden beiden Ziffern

Nachdem wir nun unsere Referenztabelle einschließlich der Konfidenzintervalle vorbereitet haben, wollen wir uns auf die beiden führenden Ziffern der "CASH_OUT"-Transaktionen konzentrieren. Dazu extrahieren wir die beiden führenden Ziffern jeder Transaktion und fügen sie als separate Spalte zum Datenrahmen aller Transaktionen hinzu:

In [ ]:
transactions_cash_out["digit_2"] = transactions_cash_out["amount"].astype(str).str[0] + transactions_cash_out["amount"].astype(str).str[1]

Überprüfen wir die extrahierten führenden Ziffern anhand der ersten 10 Zeilen des Transaktionsdatensatzes:

In [ ]:
transactions_cash_out[["amount", "digit_2"]].head(10)

In einem nächsten Schritt wollen wir die tatsächliche Wahrscheinlichkeit bestimmen, eine bestimmte Kombination von führenden Ziffern im Datensatz der "CASH_OUT"-Transaktionen zu beobachten. Dazu werden wir eine Liste aller beobachtbaren führenden Ziffern im Datensatz erstellen:

In [ ]:
benford_analysis = pd.DataFrame({"digit_2": transactions_cash_out["digit_2"].value_counts().index.map(lambda t: t.replace('.', '')).astype(np.int64).tolist()})

Als Nächstes wird gezählt, wie oft eine bestimmte Kombination von führenden Ziffern in den "CASH_OUT"-Transaktionen vorkommt:

In [ ]:
benford_analysis["count"] = transactions_cash_out["digit_2"].value_counts().tolist()

Schließlich berechnen wir die Wahrscheinlichkeit, eine bestimmte Kombination von führenden Ziffern in den "CASH_OUT"-Transaktionen zu beobachten:

In [ ]:
benford_analysis["probability"] = transactions_cash_out["digit_2"].value_counts(normalize=True).tolist()

Lassen Sie uns nun die abgeleiteten Wahrscheinlichkeiten untersuchen und überprüfen:

In [ ]:
benford_analysis

#### 4.2.3 Benford-Newcomb Soll vs. Ist Abweichungsanalyse der beiden führenden Ziffern

Zum Abschluss der Benford-Newcomb-Analyse wollen wir die ursprünglich erstellte Referenztabelle der Benford-Newcomb-Wahrscheinlichkeiten mit der tatsächlich beobachteten Wahrscheinlichkeit für eine bestimmte Kombination führender Ziffern zusammenführen. Zu diesem Zweck verwenden wir erneut die in der Bibliothek `Pandas` verfügbare Funktion `Merge`:

In [ ]:
analysis_result_double_leading_digits = benford_table.merge(benford_analysis, on="digit_2")

Nun sind wir endlich in der Lage, beide Wahrscheinlichkeiten (die erwartete Wahrscheinlichkeit nach Benford-Newcomb und die beobachtete Wahrscheinlichkeit im Datensatz) zu vergleichen und mögliche Abweichungen zu erkennen:

In [ ]:
analysis_result_double_leading_digits 

Lassen Sie uns außerdem noch einmal die von Benford-Newcomb erwartete Wahrscheinlichkeitsverteilung und die beobachteten Wahrscheinlichkeiten, die im Datensatz der "CASH_OUT"-Transaktionen verfügbar sind, visuell überprüfen:

In [ ]:
# initialise the plot 
fig, ax = plt.subplots(figsize=(15, 5))

# plot the benford probabilities 
plt.plot(analysis_result_double_leading_digits["digit_2"], analysis_result_double_leading_digits["benford"], color="red")

# plot the actual distribution of the first digit
plt.bar(analysis_result_double_leading_digits["digit_2"], analysis_result_double_leading_digits["probability"], color="green")

# plot the benford probability density
plt.fill_between(analysis_result_double_leading_digits["digit_2"], analysis_result_double_leading_digits["benford"], color="red", alpha=0.1)

# add the axis labels
plt.ylabel("[Probability]", fontsize=12)
plt.xlabel("[Leading Digits]", fontsize=12)

# format the x-tick labels
plt.xticks(range(10, 100), range(10, 100))

# rotate x-axis tick labels
plt.xticks(rotation=90)

# format the x-axis limits
plt.xlim(9.0, 100.0)

# format the y-axis limits
plt.ylim(0.0, 0.05)

# add the plot title
plt.title("Benford-Newcomb Distribution - First and Second Leading Digit", fontsize=12);

### 4.3. Detailanalyse Signifikanter Verteilungsabweichungen

Im nächsten Schritt untersuchen wir die Kombination der führenden Ziffern des Transaktionsbetrags, die im Vergleich zur Benford-Newcomb-Verteilung die größte Abweichung aufweist. Dazu berechnen wir das Delta zwischen der Benford-Newcomb-Wahrscheinlichkeit und der tatsächlich beobachtbaren Wahrscheinlichkeit für jede Kombination führender Ziffern:

In [ ]:
analysis_result_double_leading_digits["delta"] = np.abs(analysis_result_double_leading_digits["benford"] -  analysis_result_double_leading_digits["probability"])

Nun können wir die Kombinationen der Wahrscheinlichkeiten der führenden Ziffern ermitteln, die eine signifikante Abweichung aufweisen. Dazu sortieren wir den Datenrahmen entsprechend mit der Funktion `sort_values` der Bibliothek `Pandas`:

In [ ]:
analysis_result_double_leading_digits.sort_values(by=['delta'], ascending=False)

Nachfolgend werden wir die erhaltene Abweichungen entsprechend visualisieren:

In [ ]:
# initialise the plot 
fig, ax = plt.subplots(figsize=(15, 5))

# plot the actual distribution of the first digit
plt.bar(analysis_result_double_leading_digits["digit_2"], analysis_result_double_leading_digits["delta"], color="darkviolet")

# add the axis labels
plt.ylabel("[Probability Deviation]", fontsize=12)
plt.xlabel("[Leading Digits]", fontsize=12)

# format the x-tick labels
plt.xticks(range(10, 100), range(10, 100))

# rotate x-axis tick labels
plt.xticks(rotation=90)

# format the x-axis limits
plt.xlim(9.5, 100.0)

# format the y-axis limits
plt.ylim(0.0, 0.01)

# add the plot title
plt.title("Benford-Newcomb Deviation Analysis - First and Second Leading Digit", fontsize=12);

Aus der oben dargestellten Abweichungsanalyse lässt sich ein signifikanter Unterschied für die Ziffernkombinationen von **16** bis **22** ablesen. Dabei entspricht die Zahlenkombination **18** der höchsten Zahlenkombination. 

Im Folgenden werden daher alle "CASH_OUT"-Transaktionen extrahiert, die die Ziffernkombination **18** aufweisen:

In [ ]:
# set digit combination
digit = "18"

# filter corresponding cash out transactions
transactions_cash_out_18 = transactions_cash_out[transactions_cash_out["digit_2"] == digit]

Als Nächstes wollen wir uns die extrahierten Transaktionen ansehen: 

In [ ]:
transactions_cash_out_18.sort_values(by=['amount'], ascending=False)

Betrachten wir nun im Detail die Beträge der extrahierten "CASH_OUT"-Transaktionen, die die Zahlenkombination **18** aufweisen:

In [ ]:
# initialize the plot
fig, ax = plt.subplots(figsize=(15, 5))

# scatter plot of cash out transactions that exhibit a leading digit amount equal to 18
plt.scatter(transactions_cash_out_18.index, transactions_cash_out_18["amount"], color="darkviolet")

# plot unusual amount threshold
plt.axhline(y=1750000, color="r", linestyle="--", label="threshold")

# add labels of the x- and y-axis
plt.ylabel("[Amount]", fontsize=12)
plt.xlabel("[Transaction]", fontsize=12)

# format y-axis tick labels
ax.ticklabel_format(style='plain')

# hide x-ticks
plt.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)

# add plot title
plt.title("Benford-Newcomb Deviation Analysis - First and Second Leading Digit: 18", fontsize=14);

Ok, es scheint, dass ein ungewöhnliches Muster von Transaktionsbeträgen erkennbar ist.

Wenden wir einen Filter an, um alle "CASH_OUT"-Transaktionen zu ermitteln, die einem Gesamttransaktionsvolumen **gleich oder größer als ein Betrag von 1,75 Mio.** in Landeswährung entsprechen:

In [ ]:
# define the amount threshold
threshold = 1750000

# filter the cash-out transactions according to the amount threshold 
transactions_cash_out_18_large = transactions_cash_out_18[transactions_cash_out_18["amount"] >= threshold]

Lassen Sie uns eine stichprobenartige Überprüfung der extrahierten Transaktionen vornehmen: 

In [ ]:
transactions_cash_out_18_large.head(20)

Abschließend wollen wir die gefilterten Transaktionen in eine Excel-Tabelle extrahieren, um weitere stichprobenartige Prüfungen durch das Audit-Team durchzuführen. Daher werden wir in einem ersten Schritt einen Zeitstempel des Datenextrakts zu Prüfzwecken erstellen:

In [ ]:
timestamp = dt.datetime.utcnow().strftime("%Y-%m-%d_%H-%M-%S")

Jetzt extrahieren wir die gefilterten Transaktionen nach Excel in das lokale Dateisystem:

In [ ]:
# specify the filename of the excel spreadsheet
filename = str(timestamp) + " - ACA_001_benford_newcomb_18.xlsx"

# specify the target data directory of the excel spreadsheet
data_directory = os.path.join('./03_results', filename)

# extract the filtered transactions to excel
transactions_cash_out_18_large.to_excel(data_directory, header=True, index=False, sheet_name="Business_Partner_Amounts", encoding="utf-8")

## Lab Aufgaben:

Im Ihr wissen zu vertiefen empfehlen wir, die nachfolgenden Übungen zu bearbeiten:

**1. Analyse der "CASH-OUT" Transaktionen, welche die führenden Ziffern 15 und 16 aufweisen.**

> Analysieren Sie die ca. 2,2 Millionen "CASH-OUT"-Transaktionen im Hinblick auf die führenden Zahlenkombinationen "15" und "16". Folgen Sie dazu den innerhalb des Notebooks vorgestellten Verfahren. Extrahieren Sie die einzelnen Transaktionen für eine etwaige nachgelagerte Stichprobenprüfung in eine separate Excel- oder CSV-Datei.

In [ ]:
# ***************************************************
# Sie können Ihre Lösung an dieser Stelle einfügen
# ***************************************************

**2. Durchführung der Newcomb-Benford Analyse für Transaktionen des Typs "TRANSFER".**

>  Analysieren Sie die im Rahmen der Datenvalidierung extrahierten 532'909 "TRANSFER"-Transaktionen anhand von Benford-Newcomb. Befolgen Sie dabei das in den Abschnitten 4.1 bis 4.3 des Notebooks beschriebene Verfahren. Extrahieren Sie die einzelnen Transaktionen, die den Abweichungen nach Benford-Newcomb entsprechen, in eine separate Excel- oder CSV-Datei für eine entwaige nachgelagerte Stichprobenprüfung.

In [ ]:
# ***************************************************
# Sie können Ihre Lösung an dieser Stelle einfügen
# ***************************************************

### Lab Summary:

Dieses vierte Lab Notebook umfasst eine schrittweise Einführung in die Vorgehensweise Mathematisch-Statistischer Analysen im Kontext revisorischer Datenanalysen. Dabei wurde insbesondere der Teilschritt **Datenanalyse** des im Seminars vorgestellten Datenanalyseprozesses behandelt. Die vorgestellten Codebeispiele und Übungen können als Ausgangspunkt für komplexere und Ihre massgeschneiderten Analysen dienen.